## Step 1. Import libraries

In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import math 
import copy
import pickle
import gc

from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Step 1. Load and explore the data

In [3]:
TARGET = 'Response'
SEED = 94

In [4]:
print('Loading Data...')
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

submission_data = pd.read_csv('input/sample_submission.csv')

print('Data Load Successfully.')

Loading Data...
Data Load Successfully.


In [5]:
train.shape, test.shape

((11504798, 12), (7669866, 11))

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.0+ GB


In [7]:
train.describe()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,1.150480e+07,1.150480e+07,1.150480e+07,1.150480e+07,1.150480e+07,1.150480e+07,1.150480e+07,1.150480e+07,1.150480e+07
mean,5.752398e+06,3.838356e+01,9.980220e-01,2.641869e+01,4.629966e-01,3.046137e+04,1.124254e+02,1.638977e+02,1.229973e-01
std,3.321149e+06,1.499346e+01,4.443120e-02,1.299159e+01,4.986289e-01,1.645475e+04,5.403571e+01,7.997953e+01,3.284341e-01
min,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.630000e+03,1.000000e+00,1.000000e+01,0.000000e+00
25%,2.876199e+06,2.400000e+01,1.000000e+00,1.500000e+01,0.000000e+00,2.527700e+04,2.900000e+01,9.900000e+01,0.000000e+00
50%,5.752398e+06,3.600000e+01,1.000000e+00,2.800000e+01,0.000000e+00,3.182400e+04,1.510000e+02,1.660000e+02,0.000000e+00
75%,8.628598e+06,4.900000e+01,1.000000e+00,3.500000e+01,1.000000e+00,3.945100e+04,1.520000e+02,2.320000e+02,0.000000e+00
max,1.150480e+07,8.500000e+01,1.000000e+00,5.200000e+01,1.000000e+00,5.401650e+05,1.630000e+02,2.990000e+02,1.000000e+00


In [8]:
train

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,Male,48,1,6.0,0,1-2 Year,Yes,27412.0,26.0,218,0
11504794,11504794,Female,26,1,36.0,0,< 1 Year,Yes,29509.0,152.0,115,1
11504795,11504795,Female,29,1,32.0,1,< 1 Year,No,2630.0,152.0,189,0
11504796,11504796,Female,51,1,28.0,0,1-2 Year,Yes,48443.0,26.0,274,1


In [9]:
#Combine daataset for processing
train['is_train'] = 1
test['is_train'] = 0

df = pd.concat([train, test])
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,is_train
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0.0,1
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1.0,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0.0,1
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0.0,1
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0.0,1


## Step 2. Data preprocessing

In [10]:
# Check missing values
df.isnull().sum()

id                            0
Gender                        0
Age                           0
Driving_License               0
Region_Code                   0
Previously_Insured            0
Vehicle_Age                   0
Vehicle_Damage                0
Annual_Premium                0
Policy_Sales_Channel          0
Vintage                       0
Response                7669866
is_train                      0
dtype: int64

* **Age and Vehicle_Age (0.77)**:
    Strong positive correlation. Older individuals tend to have older vehicles.
* **Previously_Insured and Vehicle_Damage (-0.84)**:
    Strong negative correlation. If someone is previously insured, their vehicle is less likely to be damaged.
* **Policy_Sales_Channel and Age (-0.60)**:
    Moderate negative correlation. Younger individuals are more likely to be reached through certain sales channels.

In [11]:
def transform_categorical_features(df):
    print('Transforming categorical features..')

    gender_map = {'Male': 0, 'Female': 1}
    vehicle_age = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2} 
    vehicle_damage = {'No':0, 'Yes':1}

    df['Gender'] = df['Gender'].map(gender_map)
    df['Vehicle_Age'] = df['Vehicle_Age'].map(vehicle_age)
    df['Vehicle_Damage'] = df['Vehicle_Damage'].map(vehicle_damage)

    print("Transformed successfully.")
    return df

In [12]:
def create_additional_features(df):
    print('Creating additional features..')
    
    df['VA_PSC'] = pd.factorize(df['Vehicle_Age'].astype(str) + df['Policy_Sales_Channel'].astype(str))[0]
    df['A_VA'] = pd.factorize(df['Age'].astype(str) + df['Vehicle_Age'].astype(str))[0]
    df['PI_VD'] = pd.factorize(df['Previously_Insured'].astype(str) + df['Vehicle_Damage'].astype(str))[0]
    df['PI_V'] = pd.factorize(df['Previously_Insured'].astype(str) + df['Vintage'].astype(str))[0]
    df['PSC_A'] = pd.factorize(df['Policy_Sales_Channel'].astype(str) + df['Age'].astype(str))[0]

    return df

In [13]:
def adjust_data_types(df):
    print('Adjusting data types')
    df['Region_Code'] = df['Region_Code'].astype(int)
    df['Annual_Premium'] = df['Annual_Premium'].astype(int)
    df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype(int)
    
    return df

In [14]:
def optimize_memory_usage(df):
    print('Optimizing memory usage')
    start_mem_usage = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        if col_type.name in ['category', 'object']:
            raise ValueError(f"Column '{col}' is of type '{col_type.name}'")

        c_min = df[col].min()
        c_max = df[col].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

    end_mem_usage = df.memory_usage().sum() / 1024**2
    print(f'------ Memory usage before: {start_mem_usage:.2f} MB')
    print(f'------ Memory usage after: {end_mem_usage:.2f} MB')
    print(f'------ Reduced memory usage by {(100 * (start_mem_usage - end_mem_usage) / start_mem_usage):.1f}%')

    return df

In [15]:
def scale_features(df, features_to_scale):
    print('Scaling features...')
    min_max_scaler = MinMaxScaler()
    df[features_to_scale] = min_max_scaler.fit_transform(df[features_to_scale])
    print('Features scaled successfully.')
    return df

In [16]:
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,is_train
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0.0,1
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1.0,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0.0,1
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0.0,1
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0.0,1


In [17]:
df = transform_categorical_features(df)
df = adjust_data_types(df)  
df = create_additional_features(df)
df = optimize_memory_usage(df)                      

Transforming categorical features..
Transformed successfully.
Adjusting data types
Creating additional features..
Optimizing memory usage
------ Memory usage before: 2560.09 MB
------ Memory usage after: 713.17 MB
------ Reduced memory usage by 72.1%


In [18]:
features_to_scale = [ 'Annual_Premium', 'Vintage', 'Policy_Sales_Channel']
df = scale_features(df, features_to_scale)

df.head()    

Scaling features...
Features scaled successfully.


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,is_train,VA_PSC,A_VA,PI_VD,PI_V,PSC_A
0,0,0,21,1,35,0,1,1,0.116218,0.759259,0.612457,0.0,1,0,0,0,0,0
1,1,0,43,1,28,0,2,1,0.104702,0.154321,0.961938,1.0,1,1,1,0,1,1
2,2,1,25,1,14,1,0,0,0.065880,0.932099,0.844291,0.0,1,2,2,1,2,2
3,3,1,35,1,1,0,1,1,0.000000,0.956790,0.228374,0.0,1,3,3,0,3,3
4,4,1,36,1,15,1,1,0,0.054547,0.932099,0.982699,0.0,1,4,4,1,4,4


In [ ]:
# Compute the correlation matrix
corr = df.corr()

# Create a heatmap of the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

In [19]:
import tensorflow as tf

print("GPU Available: ", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  True


## Step 4. Split the data

In [20]:
# Split the data back into train and test sets
train = df[df['is_train'] == 1].drop(columns=['is_train'])
test = df[df['is_train'] == 0].drop(columns=['is_train'])

X_train = train.drop(columns=[TARGET])
y_train = train[TARGET]

X_test = test.drop(columns=[TARGET])

In [21]:
X_train.shape

(11504798, 16)

### Subsample the data to speed up training process

In [22]:

X_train_subsample = X_train.sample(frac=0.1, random_state=42)
y_train_subsample = y_train.sample(frac=0.1, random_state=42)


In [23]:
X_test_subsample = X_test.sample(frac = 0.1, random_state=42)

In [24]:
X_train_subsample.shape
X_test_subsample.shape

(766987, 16)

In [25]:
X_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,VA_PSC,A_VA,PI_VD,PI_V,PSC_A
0,0,0,21,1,35,0,1,1,0.116218,0.759259,0.612457,0,0,0,0,0
1,1,0,43,1,28,0,2,1,0.104702,0.154321,0.961938,1,1,0,1,1
2,2,1,25,1,14,1,0,0,0.065880,0.932099,0.844291,2,2,1,2,2
3,3,1,35,1,1,0,1,1,0.000000,0.956790,0.228374,3,3,0,3,3
4,4,1,36,1,15,1,1,0,0.054547,0.932099,0.982699,4,4,1,4,4


## Step 5. Train the model CatBoost

In [28]:
def train_and_evaluate(model, X, y, X_test, folds=5, random_state=None):
    print(f'Training {model.__class__.__name__}\n')
    
    scores = []
    feature_importances = np.zeros(X.shape[1])
    evaluation_history = []
    
    oof_pred_probs = np.zeros(X.shape[0])
    test_pred_probs = np.zeros(X_test.shape[0])
    
    skf = StratifiedKFold(n_splits=folds, random_state=random_state, shuffle=True)
    
    for fold_index, (train_index, val_index) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model_clone = copy.deepcopy(model)
        model_clone.fit(
            X_train, 
            y_train, 
            eval_set=[(X_val, y_val)], 
            early_stopping_rounds=100,
            verbose=500
        )
        
        feature_importances += model_clone.feature_importances_ / folds
        evaluation_history.append(model_clone.get_evals_result())
        
        y_pred_probs = model_clone.predict_proba(X_val)[:, 1]
        oof_pred_probs[val_index] = y_pred_probs
        
        temp_test_pred_probs = model_clone.predict_proba(X_test)[:, 1]
        test_pred_probs += temp_test_pred_probs / folds
        
        auc_score = roc_auc_score(y_val, y_pred_probs)
        scores.append(auc_score)
        
        print(f'\n--- Fold {fold_index + 1} - AUC: {auc_score:.5f}\n\n')
        
        del model_clone
        gc.collect()
    
    print(f'------ Average AUC: {np.mean(scores):.5f} ± {np.std(scores):.5f}\n\n')

    return oof_pred_probs, test_pred_probs, feature_importances, evaluation_history

In [31]:
from catboost import CatBoostClassifier

# Example best parameters for CatBoost
best_params = {
    'depth': 10,
    'learning_rate': 0.03,
    'iterations': 2000,
    'l2_leaf_reg': 3,
    'border_count': 50,
    'thread_count': -1,
    'od_type': 'Iter',
    'od_wait': 100,
    'eval_metric': 'AUC'  # Set the evaluation metric here
}

best_catboost_model = CatBoostClassifier(**best_params, random_state=94, verbose=500)

# Call train_and_evaluate function with CatBoostClassifier model
oof_pred_probs, predictions, feature_importances, evaluation_history = train_and_evaluate(best_catboost_model, X_train, y_train, X_test, folds=10, random_state=94)

Training CatBoostClassifier

0:	test: 0.8494803	best: 0.8494803 (0)	total: 1.68s	remaining: 55m 56s
500:	test: 0.8750024	best: 0.8750024 (500)	total: 15m 38s	remaining: 46m 48s
1000:	test: 0.8774629	best: 0.8774629 (1000)	total: 30m 42s	remaining: 30m 39s
1500:	test: 0.8783090	best: 0.8783090 (1500)	total: 46m 53s	remaining: 15m 35s
1999:	test: 0.8787633	best: 0.8787634 (1998)	total: 1h 2m 50s	remaining: 0us

bestTest = 0.87876342
bestIteration = 1998

Shrink model to first 1999 iterations.

--- Fold 1 - AUC: 0.87876


0:	test: 0.8499380	best: 0.8499380 (0)	total: 1.39s	remaining: 46m 23s
500:	test: 0.8755444	best: 0.8755444 (500)	total: 10m 35s	remaining: 31m 41s
1000:	test: 0.8778838	best: 0.8778838 (1000)	total: 21m 19s	remaining: 21m 17s
1500:	test: 0.8787084	best: 0.8787084 (1500)	total: 31m 45s	remaining: 10m 33s
1999:	test: 0.8790769	best: 0.8790769 (1999)	total: 42m 14s	remaining: 0us

bestTest = 0.8790768744
bestIteration = 1999


--- Fold 2 - AUC: 0.87908


0:	test: 0.8501413

In [32]:
submission = pd.DataFrame({
    'id': X_test['id'],
    'Response': predictions
})
submission.to_csv('submission.csv', index=False)
submission.head()

,id,Response
0,11504798,0.004694
1,11504799,0.438050
2,11504800,0.238449
3,11504801,0.000120
4,11504802,0.048471


## Hyperparameters tuning using Bayesian Optimization

In [25]:
from skopt import BayesSearchCV
from catboost import CatBoostClassifier

# Define the parameter grid
param_grid = {
    'depth': (4, 12),
    'learning_rate': (0.01, 0.1),
    'iterations': (500, 2000),
    'l2_leaf_reg': (1, 10),
    'border_count': (32, 255),
    'bagging_temperature': (0, 1),
    'random_strength': (0, 1)
}

# Initialize CatBoost model
catboost_model = CatBoostClassifier(eval_metric='AUC', random_state=42, verbose=100)

# Bayesian search
bayes_search = BayesSearchCV(
    estimator=catboost_model,
    search_spaces=param_grid,
    n_iter=50,  # Adjust number of iterations based on computational resources
    scoring='roc_auc',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the Bayesian search model
bayes_search.fit(X_train_subsample, y_train_subsample)

# Best parameters and score
print("Best parameters found: ", bayes_search.best_params_)
print("Best AUC found: ", bayes_search.best_score_)

ModuleNotFoundError: No module named 'importlib.metadata'